# Размещение баннеров
__Описание задачи__: международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Надо выбрать 20 таких локаций для размещения, чтобы польза была большой.

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('checkins.csv', sep='|',
                   skiprows=[0, 1],
                   skipinitialspace=True,
                   index_col=0,
                   names=['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'],
                   low_memory=False).dropna()

In [3]:
data.head()

,user_id,venue_id,latitude,longitude,created_at
id,,,,,
984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


Убедимся, что все 396634 строки с координатами считаны успешно.

In [4]:
data.shape

(396634, 5)

Оставим первые 100 000 записей для сокращения времени работы алгоритма

In [5]:
coordinates_df = data.iloc[:100000][['latitude', 'longitude']]

In [6]:
coordinates_df.shape

(100000, 2)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

In [7]:
from sklearn.cluster import MeanShift

In [8]:
model = MeanShift(bandwidth=0.1)

In [9]:
model.fit(coordinates_df)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [10]:
labels = model.labels_
centers = model.cluster_centers_

In [11]:
coordinates_df['cluster'] = labels

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [12]:
coordinates_filtered_df = coordinates_df.groupby(by='cluster').filter(lambda x: len(x) > 15)

In [14]:
banners = centers[coordinates_filtered_df['cluster'].unique()]

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

* 33.751277, -118.188740 (Los Angeles)

* 25.867736, -80.324116 (Miami)

* 51.503016, -0.075479 (London)

* 52.378894, 4.885084 (Amsterdam)

* 39.366487, 117.036146 (Beijing)

* -33.868457, 151.205134 (Sydney)

In [15]:
company_addresses = np.array([[33.751277, -118.188740], [25.867736, -80.324116],
                              [51.503016, -0.075479], [52.378894, 4.885084],
                              [39.366487, 117.036146], [-33.868457, 151.205134]])

Осталось определить 20 ближайших к баннерам центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [16]:
distances = []
for idx_off, office_crd in enumerate(company_addresses):
    for idx_ban, banner_crd in enumerate(banners):
        distance = np.linalg.norm(office_crd - banner_crd)
        distances.append((distance, (idx_off, idx_ban)))

In [17]:
distances.sort(key=lambda x: x[0])

In [18]:
print('Топ-20 ближайщих к офису баннеров:')
distances[:20]

Топ-20 ближайщих к офису баннеров:


[(0.007834758163107856, (5, 139)),
 (0.009353316185992226, (3, 270)),
 (0.022674066158385495, (1, 355)),
 (0.05005829482278787, (2, 118)),
 (0.07084773242719973, (0, 18)),
 (0.13410903336184654, (1, 165)),
 (0.16740596425035326, (1, 140)),
 (0.18887596060185083, (1, 112)),
 (0.19577945647763628, (0, 184)),
 (0.21181053682436798, (0, 93)),
 (0.2222332907317907, (0, 124)),
 (0.2713007595066735, (1, 248)),
 (0.2949788868004569, (0, 278)),
 (0.3022701186924605, (1, 135)),
 (0.30473050307840693, (0, 106)),
 (0.3148837903362732, (0, 11)),
 (0.3388104702511318, (0, 15)),
 (0.3408456533220572, (1, 134)),
 (0.37868750125029754, (0, 52)),
 (0.3867062248427277, (0, 16))]

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [19]:
nearest_banner = banners[distances[0][1][1]]

In [20]:
with open('answer.txt', 'w') as f:
    f.write(' '.join([*nearest_banner.astype('str')]))

In [21]:
!cat answer.txt

-33.86063042857143 151.20477592857145